In [1]:
# Importing the necessary libraries
import numpy as np
import pandas as pd

# Generating the datasets again using the provided code block

# Define the range of data and other parameters
num_additional_rows = 50
reporting_dates = ["Jan 1st", "April 1st", "July 1st", "Oct 1st"]
quarters = [1, 2, 3, 4]
clinic_names = [
    "Downtown Clinic", 
    "Uptown Clinic", 
    "Suburban Clinic", 
    "Riverside Clinic", 
    "Central Medical Center", 
    "Eastside Health Hub", 
    "West End Wellness Center", 
    "North Point Clinic", 
    "South Shore Medical"
]

# Generate additional data for "Patient Metrics"
np.random.seed(0)  # For reproducibility
additional_patient_data = {
    "Clinic Name": np.random.choice(clinic_names, num_additional_rows),
    "Reporting Date": np.random.choice(reporting_dates, num_additional_rows),
    "Quarter": np.random.choice(quarters, num_additional_rows),
    "Latest Report": np.random.choice([True, False], num_additional_rows),
    "Total Patients": np.random.randint(30, 150, num_additional_rows),
    "Successful Surgeries": np.random.randint(20, 120, num_additional_rows),
    "Medical Consultations": np.random.randint(20, 140, num_additional_rows),
    "Emergency Visits": np.random.randint(5, 100, num_additional_rows),
    "Patient Satisfaction Score (out of 100)": np.random.randint(60, 100, num_additional_rows)
}
additional_patient_df = pd.DataFrame(additional_patient_data)

# Generate additional data for "Operational Metrics"
choices = ["Yes", "No", "Maybe"]
efficiency_levels = ["High", "Moderate", "Low"]
additional_operational_data = {
    "Clinic Name": np.random.choice(clinic_names, num_additional_rows),
    "Reporting Date": np.random.choice(reporting_dates, num_additional_rows),
    "Quarter": np.random.choice(quarters, num_additional_rows),
    "Latest Report": np.random.choice([True, False], num_additional_rows),
    "Equipment Maintenance": np.random.choice(choices, num_additional_rows),
    "Medication Stock Available": np.random.choice(choices, num_additional_rows),
    "Cleanliness Standards Met": np.random.choice(choices, num_additional_rows),
    "Sufficient Staffing": np.random.choice(choices, num_additional_rows),
    "Patient Feedback Collected": np.random.choice(choices, num_additional_rows),
    "Operational Efficiency Level": np.random.choice(efficiency_levels, num_additional_rows)
}
additional_operational_df = pd.DataFrame(additional_operational_data)

# Initialize data and data_pt2 as empty DataFrames
data = pd.DataFrame()
data_pt2 = pd.DataFrame()

# Append the generated data to the empty DataFrames
Patient_Metric = pd.concat([data, additional_patient_df], ignore_index=True)
Operational_Metrics = pd.concat([data_pt2, additional_operational_df], ignore_index=True)

# Add a unique ID column
Operational_Metrics['RecordID'] = range(1, len(Operational_Metrics) + 1)

# Proceed with the aggregation for Patient_Metric
aggregated_patient_data = Patient_Metric.groupby(['Clinic Name', 'Quarter']).agg({
    'Total Patients': 'sum',
    'Successful Surgeries': 'sum',
    'Medical Consultations': 'sum',
    'Emergency Visits': 'sum',
    'Patient Satisfaction Score (out of 100)': 'mean'
}).reset_index()

# As for Reporting Date and Latest Report, they don't make sense to aggregate, 
# so we can simply take the most recent one for each group (assuming the data is sorted by date which isn't provided)
aggregated_patient_data['Reporting Date'] = Patient_Metric.groupby(['Clinic Name', 'Quarter'])['Reporting Date'].last().values
aggregated_patient_data['Latest Report'] = Patient_Metric.groupby(['Clinic Name', 'Quarter'])['Latest Report'].last().values

# Set 'Latest Report' to False for all rows
aggregated_patient_data['Latest Report'] = False
Operational_Metrics['Latest Report'] = False
Patient_Metric['Latest Report'] = False

# Set 'Latest Report' to True only for rows where 'Quarter' is 4
aggregated_patient_data.loc[aggregated_patient_data['Quarter'] == 4, 'Latest Report'] = True
Operational_Metrics.loc[Operational_Metrics['Quarter'] == 4, 'Latest Report'] = True
Patient_Metric.loc[Patient_Metric['Quarter'] == 4, 'Latest Report'] = True

In [2]:
# Step 1: Create a list of all combinations of clinics and quarters
all_combinations = [(clinic, quarter) for clinic in clinic_names for quarter in quarters]

# Step 2: Check which combinations are missing from the Patient_Metric dataset
missing_combinations_pt = [comb for comb in all_combinations if not ((Patient_Metric['Clinic Name'] == comb[0]) & (Patient_Metric['Quarter'] == comb[1])).any()]

# Step 3: Fill in these missing combinations with dummy or default data for Patient_Metric
for clinic, quarter in missing_combinations_pt:
    dummy_data = {
        "Clinic Name": clinic,
        "Reporting Date": reporting_dates[quarter-1],  # Use the quarter to get the reporting date
        "Quarter": quarter,
        "Latest Report": quarter == 4,
        "Total Patients": 0,
        "Successful Surgeries": 0,
        "Medical Consultations": 0,
        "Emergency Visits": 0,
        "Patient Satisfaction Score (out of 100)": 0
    }
    Patient_Metric = pd.concat([Patient_Metric, pd.DataFrame([dummy_data])], ignore_index=True)

# Step 2: Check which combinations are missing from the Operational_Metrics dataset
missing_combinations_op = [comb for comb in all_combinations if not ((Operational_Metrics['Clinic Name'] == comb[0]) & (Operational_Metrics['Quarter'] == comb[1])).any()]

# Step 3: Fill in these missing combinations with dummy or default data for Operational_Metrics
for clinic, quarter in missing_combinations_op:
    dummy_data = {
        "Clinic Name": clinic,
        "Reporting Date": reporting_dates[quarter-1],  # Use the quarter to get the reporting date
        "Quarter": quarter,
        "Latest Report": quarter == 4,
        "Equipment Maintenance": "Maybe",
        "Medication Stock Available": "Maybe",
        "Cleanliness Standards Met": "Maybe",
        "Sufficient Staffing": "Maybe",
        "Patient Feedback Collected": "Maybe",
        "Operational Efficiency Level": "Moderate"
    }
    Operational_Metrics = pd.concat([Operational_Metrics, pd.DataFrame([dummy_data])], ignore_index=True)
    Operational_Metrics['RecordID'] = range(1, len(Operational_Metrics) + 1)  # Reset RecordID to ensure uniqueness

In [3]:
# Calculate KPIs for Patient_Metric
Patient_Metric['Patient Experience Score'] = Patient_Metric['Patient Satisfaction Score (out of 100)']
Patient_Metric['Medical Efficacy Ratio'] = Patient_Metric['Successful Surgeries'] / Patient_Metric['Total Patients']
Patient_Metric['Service Demand Ratio'] = Patient_Metric['Emergency Visits'] / Patient_Metric['Medical Consultations']


In [4]:
# Define scoring system
score_mapping = {'Yes': 1, 'Maybe': 0, 'No': -1}
efficiency_mapping = {'High': 3, 'Moderate': 2, 'Low': 1}

# Score each metric
for column in ['Equipment Maintenance', 'Medication Stock Available', 'Cleanliness Standards Met', 'Sufficient Staffing', 'Patient Feedback Collected']:
    Operational_Metrics[column + ' Score'] = Operational_Metrics[column].map(score_mapping)

Operational_Metrics['Operational Efficiency Score'] = Operational_Metrics['Operational Efficiency Level'].map(efficiency_mapping)

In [5]:
aggregated_patient_data.loc[aggregated_patient_data['Quarter'] == 4].count()

Clinic Name                                8
Quarter                                    8
Total Patients                             8
Successful Surgeries                       8
Medical Consultations                      8
Emergency Visits                           8
Patient Satisfaction Score (out of 100)    8
Reporting Date                             8
Latest Report                              8
dtype: int64

In [6]:
aggregated_patient_data

,Clinic Name,Quarter,Total Patients,Successful Surgeries,Medical Consultations,Emergency Visits,Patient Satisfaction Score (out of 100),Reporting Date,Latest Report
0,Central Medical Center,1,54,105,105,80,82.000000,July 1st,False
1,Central Medical Center,2,134,71,136,5,88.000000,Oct 1st,False
2,Central Medical Center,3,68,51,89,77,73.000000,Jan 1st,False
3,Central Medical Center,4,46,52,54,88,95.000000,Oct 1st,True
4,Downtown Clinic,2,139,96,108,35,74.000000,Jan 1st,False
5,Downtown Clinic,3,199,192,266,176,74.000000,Jan 1st,False
6,Downtown Clinic,4,309,208,302,236,74.666667,Oct 1st,True
7,Eastside Health Hub,2,309,189,200,165,87.333333,Oct 1st,False
8,Eastside Health Hub,3,225,107,170,127,74.500000,Oct 1st,False
9,Eastside Health Hub,4,105,107,20,97,64.000000,July 1st,True


In [7]:
Operational_Metrics

,Clinic Name,Reporting Date,Quarter,Latest Report,Equipment Maintenance,Medication Stock Available,Cleanliness Standards Met,Sufficient Staffing,Patient Feedback Collected,Operational Efficiency Level,RecordID,Equipment Maintenance Score,Medication Stock Available Score,Cleanliness Standards Met Score,Sufficient Staffing Score,Patient Feedback Collected Score,Operational Efficiency Score
0,Riverside Clinic,Jan 1st,3,False,Maybe,No,Maybe,No,Yes,Moderate,1,0,-1,0,-1,1,2
1,West End Wellness Center,July 1st,1,False,Yes,No,No,Yes,Maybe,Moderate,2,1,-1,-1,1,0,2
2,Uptown Clinic,April 1st,4,True,No,Yes,Yes,Yes,Maybe,Low,3,-1,1,1,1,0,1
3,North Point Clinic,July 1st,4,True,No,Maybe,No,Maybe,No,High,4,-1,0,-1,0,-1,3
4,Central Medical Center,July 1st,4,True,Maybe,No,Maybe,Maybe,Maybe,High,5,0,-1,0,0,0,3
5,Suburban Clinic,April 1st,1,False,Maybe,No,No,Yes,Maybe,Moderate,6,0,-1,-1,1,0,2
6,Downtown Clinic,Oct 1st,2,False,Yes,No,Maybe,Maybe,Maybe,Moderate,7,1,-1,0,0,0,2
7,South Shore Medical,Oct 1st,4,True,Maybe,No,No,Yes,Maybe,Low,8,0,-1,-1,1,0,1
8,Suburban Clinic,Jan 1st,4,True,No,Maybe,No,Yes,Maybe,High,9,-1,0,-1,1,0,3
9,North Point Clinic,Jan 1st,3,False,Maybe,Yes,Maybe,Maybe,Yes,High,10,0,1,0,0,1,3


In [8]:
Patient_Metric

,Clinic Name,Reporting Date,Quarter,Latest Report,Total Patients,Successful Surgeries,Medical Consultations,Emergency Visits,Patient Satisfaction Score (out of 100),Patient Experience Score,Medical Efficacy Ratio,Service Demand Ratio
0,Eastside Health Hub,Jan 1st,2,False,97,23,54,27,92,92,0.237113,0.500000
1,Downtown Clinic,Jan 1st,2,False,139,96,108,35,74,74,0.690647,0.324074
2,Riverside Clinic,April 1st,1,False,41,55,53,22,82,82,1.341463,0.415094
3,Riverside Clinic,Oct 1st,2,False,116,106,25,75,88,88,0.913793,3.000000
4,North Point Clinic,Jan 1st,2,False,107,81,124,76,80,80,0.757009,0.612903
5,Riverside Clinic,April 1st,2,False,139,89,56,23,78,78,0.640288,0.410714
6,Eastside Health Hub,July 1st,4,True,105,107,20,97,64,64,1.019048,4.850000
7,Suburban Clinic,July 1st,1,False,86,63,95,48,82,82,0.732558,0.505263
8,Central Medical Center,Oct 1st,4,True,46,52,54,88,95,95,1.130435,1.629630
9,North Point Clinic,Jan 1st,2,False,54,31,89,54,79,79,0.574074,0.606742


In [9]:
# Exporting the DataFrames to Excel

excel_filepath = "Clinic Metrics.xlsx"
with pd.ExcelWriter(excel_filepath) as writer:
    aggregated_patient_data.to_excel(writer, sheet_name='Aggregated_Patient_Data', index=False)
    Operational_Metrics.to_excel(writer, sheet_name='Operational_Metrics', index=False)
    Patient_Metric.to_excel(writer, sheet_name='Patient_Metric', index=False)